Import all the packages we need

In [4]:
from typing import List

import numpy as np
import tensorflow as tf
import h5py
import os
tensorflow.python.keras.utils.np_utils import to_categorical
from keras.src.backend.tensorflow.sparse import zeros_int8
from tensorflow import Tensor
from tensorflow import Tensor
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from typing import Tuple, Dict, Any, Sequence

from collections import defaultdict
import tensorflow as tf



Import the datasets, and extract the information we are going to use (sub_byte_in and byte 0 in our test case)


In [71]:
#start with train
dataset_name = "train"
shard = []
attack_byte = 0
attack_point = "sub_bytes_in"
max_trace_length = 5000
num_traces = 256
num_traces_per_shard = 256
full_key = False
x_List: List[Tensor] = []
y_List: List[Tensor] = []

#need to open the file to read any info
with (h5py.File(f"{dataset_name}.hdf5", "r")) as f:
    for group in f.keys():
        group_name = f[group]

        x_shard = group_name["traces"][:num_traces_per_shard, :5000, :]
        x_shard = tf.convert_to_tensor(x_shard, dtype="float32")

        y_shard = group_name["sub_bytes_in"][attack_byte]
        y_shard = y_shard[:num_traces_per_shard]
        y_shard = tf.keras.utils.to_categorical(y_shard, 256)
        y_shard = tf.convert_to_tensor(y_shard, dtype="uint8")


        x_List.append(x_shard)
        y_List.append(y_shard)
    x: Tensor = tf.concat(x_List, axis=0)
    y: Tensor = tf.concat(y_List, axis=0)
x_train = x
y_train = y

In [72]:
#For test dataset, which should be scaled down as it is 1:1 to the train set atm
dataset_name = "test"
shard = []
attack_byte = 0
attack_point = "sub_bytes_in"
max_trace_length = 5000
num_traces = 256 
num_traces_per_shard = 256
full_key = False
x_List: List[Tensor] = []
y_List: List[Tensor] = []

#need to open the file to read any info
with (h5py.File(f"{dataset_name}.hdf5", "r")) as f:
    for group in f.keys():
        group_name = f[group]

        x_shard = group_name["traces"][:num_traces_per_shard, :5000, :]
        x_shard = tf.convert_to_tensor(x_shard, dtype="float32")

        y_shard = group_name["sub_bytes_in"][attack_byte]
        y_shard = y_shard[:num_traces_per_shard]
        y_shard = tf.keras.utils.to_categorical(y_shard, 256)
        y_shard = tf.convert_to_tensor(y_shard, dtype="uint8")


        x_List.append(x_shard)
        y_List.append(y_shard)
    x: Tensor = tf.concat(x_List, axis=0)
    y: Tensor = tf.concat(y_List, axis=0)
x_test = x
y_test = y

Create the resnet model we want to train(cannot use pretrained model from scaaml as it uses depreciated methods and just wont work when we try) but we use the structure



In [73]:
def block(x: Tensor,
          filters: int,
          kernel_size: int = 3,
          strides: int = 1,
          conv_shortcut: bool = False,
          activation: str = "relu") -> Tensor:
    """Residual block with pre-activation
    From: https://arxiv.org/pdf/1603.05027.pdf

    Args:
        x: input tensor.
        filters (int): filters of the bottleneck layer.

        kernel_size(int, optional): kernel size of the bottleneck layer.
        defaults to 3.

        strides (int, optional): stride of the first layer.
        defaults to 1.

        conv_shortcut (bool, optional): Use convolution shortcut if True,
        otherwise identity shortcut. Defaults to False.

        use_batchnorm (bool, optional): Use batchnormalization if True.
        Defaults to True.

        activation (str, optional): activation function. Defaults to "relu".

    Returns:
        Output tensor for the residual block.
    """

    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    if conv_shortcut:
        shortcut = layers.Conv1D(4 * filters, 1, strides=strides)(x)
    else:
        if strides > 1:
            shortcut = layers.MaxPooling1D(1, strides=strides)(x)
        else:
            shortcut = x

    x = layers.Conv1D(filters, 1, use_bias=False, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    x = layers.Conv1D(filters,
                      kernel_size,
                      strides=strides,
                      use_bias=False,
                      padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(activation)(x)

    x = layers.Conv1D(4 * filters, 1)(x)
    x = layers.Add()([shortcut, x])
    return x


# pylint: disable=too-many-positional-arguments
def stack(x: Tensor,
          filters: int,
          blocks: int,
          kernel_size: int = 3,
          strides: int = 2,
          activation: str = "relu") -> Tensor:
    """A set of stacked residual blocks.
    Args:
        filters (int): filters of the bottleneck layer.

        blocks (int): number of conv blocks to stack.

        kernel_size(int, optional): kernel size of the bottleneck layer.
        defaults to 3.

        strides (int, optional): stride used in the last block.
        defaults to 2.

        conv_shortcut (bool, optional): Use convolution shortcut if True,
        otherwise identity shortcut. Defaults to False.

        activation (str, optional): activation function. Defaults to "relu".

    Returns:
        tensor:Output tensor for the stacked blocks.
  """
    x = block(x,
              filters,
              kernel_size=kernel_size,
              activation=activation,
              conv_shortcut=True)
    for _ in range(2, blocks):
        x = block(x, filters, kernel_size=kernel_size, activation=activation)
    x = block(x, filters, strides=strides, activation=activation)
    return x

#endrer fra Model[Any, Any] til bare model
# pylint: disable=C0103
def Resnet1D(input_shape: Tuple[int, ...], attack_point: str):
    #mdl_cfg: Dict[str, Any], optim_cfg: Dict[str, Any]) -> Model:
    del attack_point  # unused

    #pool_size = mdl_cfg["initial_pool_size"]
    #filters = mdl_cfg["initial_filters"]
    #block_kernel_size = mdl_cfg["block_kernel_size"]
   # activation = mdl_cfg["activation"]
    #dense_dropout = mdl_cfg["dense_dropout"]
    #num_blocks = [
    #    mdl_cfg["blocks_stack1"], mdl_cfg["blocks_stack2"],
   #     mdl_cfg["blocks_stack3"], mdl_cfg["blocks_stack4"]
  #  ]
#manually adding the parameters as i have not figured out how to read json into the model properly yet.
    pool_size = 4
    filters = 8
    block_kernel_size = 3
    activation = "relu"
    dense_dropout = 0.1
    num_blocks = [
        3, 4,
        4, 3
    ]

    inputs = layers.Input(shape=input_shape)
    x = inputs

    # stem
    x = layers.MaxPool1D(pool_size=pool_size)(x)

    # trunk: stack of residual block
    for block_idx in range(4):
        filters *= 2
        x = stack(x,
                  filters,
                  num_blocks[block_idx],
                  kernel_size=block_kernel_size,
                  activation=activation)

    # head model: dense
    x = layers.GlobalAveragePooling1D()(x)
    for _ in range(1):
        x = layers.Dropout(dense_dropout)(x)
        x = layers.Dense(256)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)

    outputs = layers.Dense(256, activation="softmax")(x)

    model: Model[Any, Any] = Model(inputs=inputs, outputs=outputs)
    model.summary()

    #if get_num_gpu() > 1:
        #lr = optim_cfg["multi_gpu_lr"]
        #lr = 0.001

    #else:
        #lr = optim_cfg["lr"]
    lr = 0.001
    model.compile(loss=["categorical_crossentropy"],
                  metrics=["acc"],
                  optimizer=Adam(lr))
    return model

FileNotFoundError: [Errno 2] No such file or directory: 'notebooks\\stm32f0_config.json'

In [75]:
sub_in_0_byte_cnn_scaaml = Resnet1D((5000, 1), "sub_bytes_in")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 5000, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_10    │ (None, 1250, 1)   │          0 │ input_layer_2[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1250, 1)   │          4 │ max_pooling1d_10… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_86       │ (None, 1250, 1)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_93 (Conv1D)  │ (None, 1250, 16)  │         16 │ activation_86[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1250, 16)  │         64 │ conv1d_93[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_87       │ (None, 1250, 16)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_94 (Conv1D)  │ (None, 1250, 16)  │        768 │ activation_87[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1250, 16)  │         64 │ conv1d_94[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_88       │ (None, 1250, 16)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_92 (Conv1D)  │ (None, 1250, 64)  │        128 │ activation_86[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_95 (Conv1D)  │ (None, 1250, 64)  │      1,088 │ activation_88[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_28 (Add)        │ (None, 1250, 64)  │          0 │ conv1d_92[0][0],  │
│                     │                   │            │ conv1d_95[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1250, 64)  │        256 │ add_28[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_89       │ (None, 1250, 64)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_96 (Conv1D)  │ (None, 1250, 16)  │      1,024 │ activation_89[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 1250, 16)  │         64 │ conv1d_96[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_90       │ (None, 1250, 16)  │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 1,122,388 (4.28 MB)

 Trainable params: 1,113,106 (4.25 MB)

 Non-trainable params: 9,282 (36.26 KB)

Train it on the correct data, kan være lurt å gjøre før man skal predicte...

In [104]:
input_shape = x_train.shape[1:]

In [112]:

from keras.callbacks import ModelCheckpoint, TensorBoard
model_name = 'sub_bytes_in'
# Definer callback-listen
cb = [
    ModelCheckpoint(filepath=model_name +".keras", monitor="val_loss", mode="min", save_best_only=True),
    TensorBoard(log_dir="logs/", update_freq="batch")
]

# Tren modellen
history = sub_in_0_byte_cnn_scaaml.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=1, epochs=32, callbacks=cb)


Epoch 1/32
 677/2048 ━━━━━━━━━━━━━━━━━━━━ 9:18 407ms/step - acc: 0.0101 - loss: 5.4010

KeyboardInterrupt: 

We need to load our attack traces, they are close to the test and train data, but the key is random every time. ATM we do not control for repeat keys!

In [91]:
import numpy.typing as npt

def ap_preds_to_key_preds(preds: npt.NDArray[np.generic],
                          plaintexts: npt.NDArray[np.generic],
                          attack_point: str) -> npt.NDArray[np.generic]:
    "Convert attack points predictions to key byte prediction"

    preds = np.array(preds)

    if attack_point == "key":
        return preds

    elif attack_point == "sub_bytes_in":
        key_probas = []
        class_ids = np.arange(256)
        for idx, pred in enumerate(preds):
            kv = class_ids ^ plaintexts[idx]
            key_probas.append(np.take(pred, kv))
        return np.array(key_probas)

    elif attack_point == "sub_bytes_out":
        key_probas = []
        for idx, pred in enumerate(preds):
            # ! danger zone. change at your own peril, Here be dragons.
            kidxs = SBREV ^ plaintexts[idx]
            probas = np.zeros(256)
            for pidx, kidx in enumerate(kidxs):
                probas[kidx] = pred[pidx]
            key_probas.append(probas)

        return np.array(key_probas)
    else:
        raise ValueError("Invalid attack point")

def from_categorical(predictions: Sequence[npt.ArrayLike]) -> List[np.intp]:
    "reverse of categorical"
    # note: doing it as a list is significantly faster than a single argmax
    return [np.argmax(p) for p in predictions]

Test it


In [98]:
from tensorflow.keras import metrics
from tqdm import tqdm
arr = np.arange(256)

NUM_TRACES = 10  # maximum number of traces to use to recover a given key byte. 10 is already overkill
correct_prediction_rank = defaultdict(list)
y_pred = []
y_true = []
model_metrics = {"acc": metrics.Accuracy()}
#for shard in tqdm(arr, desc='Recovering bytes', unit='shards'):
   # keys, pts, x, y = load_shard(shard, ATTACK_BYTE, ATTACK_POINT, TRACE_LEN, num_traces=NUM_TRACES)
dataset_name = "attack"
shard = []
attack_byte = 0
attack_point = "sub_bytes_in"
max_trace_length = 5000
num_traces = 10

full_key = False
x_List: List[Tensor] = []
y_List: List[Tensor] = []
print("hello")
    #need to open the file to read any info
with (h5py.File(f"{dataset_name}.hdf5", "r")) as f:
    print("opening file")
    i = 0
    for group in f.keys():

        group_name = f[group]
        print("reading group")

        x_shard = group_name["traces"][:num_traces, :5000, :]
        x_shard = tf.convert_to_tensor(x_shard, dtype="float32")
        print("done with x")
        y_shard = group_name["sub_bytes_in"][attack_byte]
        y_shard = y_shard[:num_traces]
        y_shard = tf.keras.utils.to_categorical(y_shard, 256)
        y_shard = tf.convert_to_tensor(y_shard, dtype="uint8")
        print("done with y")

        k = group_name["key"][attack_byte][:num_traces]
        print("done with k")

        pts = group_name["pts"][attack_byte][:num_traces]
        print("done with pts")

                # prediction
        predictions = sub_in_0_byte_cnn_scaaml.predict(x_shard)
        print(predictions.shape)
                # computing byte prediction from intermediate predictions
        key_preds = ap_preds_to_key_preds(predictions, pts, attack_point)

        c_preds = from_categorical(predictions)
        c_y = from_categorical(y_shard)
                # metric tracking
        for metric in model_metrics.values():
            metric.update_state(c_y, c_preds)
                # for the confusion matrix
        y_pred.extend(c_preds)
        y_true.extend(c_y)

            # accumulating probabilities and checking correct guess position.
            # if all goes well it will be at position 0 (highest probability)
            # see below on how to use for the real attack


        key = k[0] # all the same in the same shard - not used in real attack
        vals = np.zeros((256))
        for trace_count, kp in enumerate(key_preds):
            vals = vals  + np.log10(kp + 1e-22)
            guess_ranks = (np.argsort(vals, )[-256:][::-1])
            byte_rank = list(guess_ranks).index(key)
            correct_prediction_rank[trace_count].append(byte_rank)


hello
opening file
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
(10, 256)
reading group
done with x
done with y
done with k
done with pts
1/1 ━━━━━━━━━━━━━━

In [99]:
print("Accuracy: %.2f" % model_metrics['acc'].result())


Accuracy: 0.00


Fetching info from a attack set will look like this